# WAVES Summer School 2025 (Gandia)

This Jupyter Notebook has been designed to be run in [Google Colab](https://colab.research.google.com/). With this purpose the first cell install [NGSolve](https://ngsolve.org/) related packages in a clean machine (if they have not been previously installed). Typically, this installation takes less than 2 minutes.

In [ ]:
# Install NGSolve
try:
    import ngsolve
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/ngsolve-install-release-complex.sh" -O "/tmp/ngsolve-install.sh" && bash "/tmp/ngsolve-install.sh"

## Linear Acoustics in the time-harmonic setting: Pressure formulation in $H^1$

This notebook illustrates the numerical solution of the wave equation for harmonic excitation using the so called [Finite Element Method](https://jschoeberl.github.io/iFEM/intro.html) (FEM). The method aims at an approximate solution by subdividing the area of interest into smaller parts with simpler geometry, linking these parts together and applying methods from the calculus of variations to solve the problem numerically. The FEM is a well established method for the numerical approximation of the solution of partial differential equations (PDEs). The solutions of PDEs are often known analytically only for rather simple geometries. FEM based simulations allow to gain insights into other more complex cases.

## Variational Formulation

The FEM is based on expressing the partial differential equation (PDE) to be solved in its variational or weak formulation.

## Numerical Solution

The numerical solution of the variational problem is based on [NGSolve](https://ngsolve.org/), an open-source framework for numerical solution of PDEs.
Its high-level Python interface is used in the following to define the problem and compute its solution.
The implementation is based on the variational formulation derived above. The definition of the problem in NGsolve is very close to the mathematical formulation of the problem. A function `FEM_pressure(...)` is defined to analyze the convergence beahviour and the optimal strategy: $h$ or $p$-refinement.

In [ ]:
# Libraries 
import time
import numpy as np
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

# Geometry
L = 1.0

# Primary geometric objects
domain_down = Rectangle(L, L/2).Face()
domain_up = MoveTo(0, L/2).Rectangle(L, L/2).Face()

# Domain and boundary tags for the upper part
domain_up.faces.name = "up"
domain_up.faces.col = (0, 1, 0)  # Green color for faces
domain_up.mat("up")
domain_up.edges.Min(X).name = "left"
domain_up.edges.Min(X).col = (1, 0, 0) # magenta
domain_up.edges.Max(Y).name = "top"
domain_up.edges.Max(Y).col = (1, 0, 0) # magenta
domain_up.edges.Max(X).name = "right"
domain_up.edges.Max(X).col = (1, 0, 0) # magenta
# Domain and boundary tags for the lower part
domain_down.faces.name = "down"
domain_down.faces.col = (0, 0, 1)  # Blue color for faces
domain_down.mat("down")
domain_down.edges.Min(Y).name = "bottom"
domain_down.edges.Min(Y).col = (1, 0, 0) # magenta
domain_down.edges.Min(X).name = "left"
domain_down.edges.Min(X).col = (1, 0, 0) # magenta
domain_down.edges.Max(X).name = "right"
domain_down.edges.Max(X).col = (1, 0, 0) # magenta

# Create the domain
domain = Glue([domain_up, domain_down])

## Each edge is colored
Draw(domain, height="3vh")

In [ ]:
# Define the mesh
h_size = 0.05
mesh = Mesh(OCCGeometry(domain, dim=2).GenerateMesh(maxh=h_size, quad_dominated=False))
Draw(mesh, height="3vh")

Exact solution and boundary data for getting a solution in closed form

In [ ]:
# Physical parameters
f = 500 # Frequency in Hz
rho1 = 1.0 # Density in kg/m^3 (inner)
rho2 = 0.5 # Density in kg/m^3 (outer)
c1 = 343.0 # Speed of sound in m/s
c2 = 300.0 # Speed of sound in m/s
# Coefficients for mass and sound speed
rho = mesh.MaterialCF({'up': rho1,  'down': rho2})
c = mesh.MaterialCF({'up': c1,  'down': c2})

# Pressure: plane wave solution at oblique incidence
theta = np.pi/4  # Angle of incidence in radians
omega = 2*np.pi*f # Angular frequency

# Wavenumbers
k1x = omega/c1*np.sin(theta)  # x-component of the wave vector
k1y = omega/c1*np.cos(theta)  # y-component of the wave vector
k2y = np.sqrt((omega/c2)**2 - k1x**2)  # y-component of the wave vector in the down domain

# Surface impedance between the two domains
Z1 = rho1*c1/np.cos(theta)  # Acoustic impedance in the up domain
Z2 = rho2*c2*(omega/c2)/k2y  # Acoustic impedance in the down domain
R = (Z2-Z1)/(Z1+Z2) # reflection coefficient
T = 2*Z2/(Z1+Z2) # transmission coefficient

# Pressure field in the entire domain
pex1 = 1.0*exp(1j*(-k1x*x-k1y*(y-L/2))) + R*exp(1j*(-k1x*x+k1y*(y-L/2))) # pressure in the up domain
pex2 = T*exp(1j*(-k1x*x-k2y*(y-L/2))) # pressure in the down domain
pex = IfPos(y - L/2, pex1, pex2)

# Gradient of the exact pressure field
grad_pex1 = 1j*(CF((-k1x,-k1y))*exp(1j*(-k1x*x-k1y*(y-L/2))) + CF((-k1x,k1y))*R*exp(1j*(-k1x*x+k1y*(y-L/2))))  # displacement in the up domain
grad_pex2 = 1j*CF((-k1x,-k2y))*T*exp(1j*(-k1x*x-k2y*(y-L/2))) # displacement in the down domain
grad_pex = mesh.MaterialCF({'up': grad_pex1,  'down': grad_pex2})

# Displacement field in the entire domain
uex1 = 1/(omega**2*rho1)*grad_pex1  # displacement in the up domain
uex2 = 1/(omega**2*rho2)*grad_pex2  # displacement in the down domain
uex = mesh.MaterialCF({'up': uex1,  'down': uex2})


# Plot the exact solution
Draw(pex, mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})
Draw(uex[0], mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})
Draw(uex[1], mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})

# Boundary data for the pressure in the interior boundary
p_bnd = pex

### Finite element computation using scalar-valued Lagrange elements

In [ ]:
# Compute the Finite Element approximation
def FEM_pressure(mesh, order_FE, f, rho, c, p_bnd):

    # Angular frequency
    omega= 2*np.pi*f

    # Lagrange finite element with complex values to approximate the pressure field
    Q = H1(mesh, order=order_FE, complex=True, dirichlet = "top|bottom|left|right")

    # Trial and test functions
    p, q = Q.TnT()

    # Variational formulation: Bilinear form
    a_bilinear = BilinearForm(Q, symmetric=False)

    # Contribution in the air and the impedance boundary
    a_bilinear += 1.0/rho*grad(p)*grad(q)*dx - omega**2/(rho*c**2)*p*q*dx

    # Linear form
    f_linear = LinearForm(Q)

    # Assembly the FEM matrices and the right-hand side
    a_bilinear.Assemble()
    f_linear.Assemble()

    # Allocate the solution vector with the prescribed Dirichlet boundary conditions
    gfp = GridFunction(Q)
    gfp.Set(p_bnd, definedon=mesh.Boundaries("top|bottom|left|right"))

    # Solve the linear system
    precond = Preconditioner(a_bilinear,"direct") # sparse direct solver (UMFPACK)
    solvers.BVP(bf=a_bilinear, lf=f_linear, gf=gfp, pre=precond, print=False)

    return gfp, Q

Function to compute the RMS errors and the errors associated with the acoustic energy of the system

In [ ]:
# Compute the root-mean square (RMS) relative error
def compute_error(rho, c, gfp, pex, grad_pex, Q):

    # RMS of the exact solution (pressure)
    RMS_p_ex = sqrt(Integrate(1.0/(rho*c**2)*InnerProduct(pex, pex), Q.mesh, order=2*Q.globalorder))

    # RMS absolute error between the exact and the Finite Element approximation (pressure)
    error_RMS_p = sqrt(Integrate(1.0/(rho*c**2)*InnerProduct(gfp-pex, gfp-pex), Q.mesh, order=2*Q.globalorder))

    # RMS of the exact solution (velocity)
    RMS_v_ex = sqrt(Integrate(1.0/rho/omega**2*InnerProduct(grad_pex, grad_pex), Q.mesh, order=2*Q.globalorder))

    # RMS absolute error between the exact and the Finite Element approximation (velocity)
    error_RMS_v = sqrt(Integrate( 1.0/rho/omega**2*InnerProduct(grad(gfp)-grad_pex, grad(gfp)-grad_pex), Q.mesh, order=2*Q.globalorder))

    # Energy associated to the exact solution
    energy_ex = sqrt(RMS_p_ex**2 + RMS_v_ex**2)

    # Energy error
    error_energy = sqrt(error_RMS_p**2 + error_RMS_v**2)

    return error_RMS_p.real/RMS_p_ex.real, error_RMS_v.real/RMS_v_ex.real, error_energy.real/energy_ex.real

### Study of convergence

Let's check the order of convergence of this finite element discretization computing the error for sucesive refinements of the mesh

In [ ]:
# Finite element order
order_FE = np.arange(1, 5)  # Orders from 1 to 4
h_size = np.array([0.4, 0.2, 0.1, 0.05])  # Mesh sizes

# Compute the Finite Element approximation and the error for different mesh sizes and orders
error_RMS_p = np.zeros((len(h_size), len(order_FE)))
error_RMS_v = np.zeros((len(h_size), len(order_FE)))
error_energy = np.zeros((len(h_size), len(order_FE)))
wall_time = np.zeros((len(h_size), len(order_FE)))

# Loop over mesh sizes and finite element orders
for i, hs in enumerate(h_size):
    mesh = Mesh(OCCGeometry(domain, dim=2).GenerateMesh(maxh=hs, quad_dominated=False))
    for j, order in enumerate(order_FE):
        mesh.Curve(int(order))
        start_wall = time.time()
        gfp, Q = FEM_pressure(mesh, int(order), f, rho, c, p_bnd)
        wall_time[i, j] = time.time() - start_wall
        error_RMS_p[i, j], error_RMS_v[i, j], error_energy[i, j] = compute_error(rho, c, gfp, pex, grad_pex, Q)
        # Print the errors
        print(f"Order {order}, Mesh size {hs}: RMS Error (Pressure) = {error_RMS_p[i, j]:.4e}, RMS Error (Velocity) = {error_RMS_v[i, j]:.4e}, Energy Error = {error_energy[i, j]:.4e}")

# Plot last pressure field computed
Draw(gfp, mesh, height="3vh", animate_complex=True, settings = {"Colormap": {"ncolors": 20}})

Compute the order of convergence of the FEM discretizations

In [ ]:
# Get order of convergence for each order_FE
for j, order in enumerate(order_FE):
    mRMS_p, _ = np.polyfit(np.log(h_size[1:]), np.log(error_RMS_p[1:, j]), 1)
    mRMS_v, _ = np.polyfit(np.log(h_size[1:]), np.log(error_RMS_v[1:, j]), 1)
    print(f"Order {order}: RMS Error convergence rate (pressure) = {mRMS_p:.1f}, RMS Error convergence rate (velocity) = {mRMS_v:.1f}")
    mEnergy, _ = np.polyfit(np.log(h_size[1:]), np.log(error_energy[1:, j]), 1)
    print(f"Order {order}: Energy Error convergence rate = {mEnergy:.1f}")

Check the numerical errors and observe the FEM convergence

In [ ]:
# Plot the convergence results with tag the order in the legend
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.loglog(h_size*f/c1.real, 100*error_RMS_p, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('h/λ')
plt.ylabel('RMS Rel. Error (%)')
plt.title('RMS Rel. Error (Pressure) vs h/λ')
plt.grid(True)
plt.legend()
plt.subplot(1, 3, 2)
plt.loglog(h_size*f/c1.real, 100*error_RMS_v, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('h/λ')
plt.ylabel('RMS Rel. Error (%)')
plt.title('RMS Rel. Error (Velocity) vs h/λ')
plt.grid(True)
plt.legend()
plt.subplot(1, 3, 3)
plt.loglog(h_size*f/c1.real, 100*error_energy, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('h/λ')
plt.ylabel('Energy Rel. Error (%)')
plt.title('Energy Rel. Error vs h/λ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



Checking the optimal strategy: refine the mesh or increasing the FE order?

In [ ]:
# Plot the CPU and wall time vs energy relative error with tag the order in the legend
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.loglog(100*error_energy, wall_time, marker='o', label=[f"Order {order}" for order in order_FE])
plt.xlabel('Energy Relative Error (%)')
plt.ylabel('CPU Time (s)')
plt.title('CPU Time vs Energy Relative Error')
plt.grid(True)
plt.legend()
plt.subplot(1, 2, 2)
plt.loglog(100*error_energy.T, wall_time.T, marker='o', label=[f"h={hs}" for hs in h_size])
plt.xlabel('Energy Relative Error (%)')
plt.ylabel('CPU Time (s)')
plt.title('CPU Time vs Energy Relative Error')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Write to a csv file the wall time and the RMS and energy errors

In [ ]:
# Write to a CSV file the errors computed with the RMS and energy norms
import pandas as pd
# Create a DataFrame with the errors and wall time
df_errors = pd.DataFrame({
    'Mesh Size (h)': np.tile(h_size, len(order_FE)),
    'Order': np.repeat(order_FE, len(h_size)),
    'RMS Error pressure': error_RMS_p.flatten(),
    'RMS Error velocity': error_RMS_v.flatten(),
    'Energy Error': error_energy.flatten(),
    'Wall Time (s)': wall_time.flatten()
})
# Save the DataFrame to a CSV file
df_errors.to_csv('H1-pressure.csv', index=False)


**Copyright**

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).